# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f778f0fa9d0>
├── 'a' --> tensor([[ 0.4277,  0.1447,  0.2535],
│                   [-0.7681,  0.2668,  1.4102]])
└── 'x' --> <FastTreeValue 0x7f778f0fac10>
    └── 'c' --> tensor([[-2.1334,  2.4800,  1.1582,  0.9062],
                        [ 0.3769, -1.3441,  0.0103, -0.9183],
                        [ 1.2914,  0.7107, -0.4273, -0.5815]])

In [4]:
t.a

tensor([[ 0.4277,  0.1447,  0.2535],
        [-0.7681,  0.2668,  1.4102]])

In [5]:
%timeit t.a

85.9 ns ± 2.03 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f778f0fa9d0>
├── 'a' --> tensor([[-1.1664,  1.3488,  0.4726],
│                   [-0.6918, -0.4377,  0.4121]])
└── 'x' --> <FastTreeValue 0x7f778f0fac10>
    └── 'c' --> tensor([[-2.1334,  2.4800,  1.1582,  0.9062],
                        [ 0.3769, -1.3441,  0.0103, -0.9183],
                        [ 1.2914,  0.7107, -0.4273, -0.5815]])

In [7]:
%timeit t.a = new_value

78.3 ns ± 1.2 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4277,  0.1447,  0.2535],
               [-0.7681,  0.2668,  1.4102]]),
    x: Batch(
           c: tensor([[-2.1334,  2.4800,  1.1582,  0.9062],
                      [ 0.3769, -1.3441,  0.0103, -0.9183],
                      [ 1.2914,  0.7107, -0.4273, -0.5815]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4277,  0.1447,  0.2535],
        [-0.7681,  0.2668,  1.4102]])

In [11]:
%timeit b.a

70.3 ns ± 1.27 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1037,  1.0359,  0.0246],
               [ 0.0638,  1.4118,  0.4258]]),
    x: Batch(
           c: tensor([[-2.1334,  2.4800,  1.1582,  0.9062],
                      [ 0.3769, -1.3441,  0.0103, -0.9183],
                      [ 1.2914,  0.7107, -0.4273, -0.5815]]),
       ),
)

In [13]:
%timeit b.a = new_value

640 ns ± 26.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.07 µs ± 27.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.4 µs ± 362 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

265 µs ± 9.52 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

255 µs ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f76b4947bb0>
├── 'a' --> tensor([[[ 0.4277,  0.1447,  0.2535],
│                    [-0.7681,  0.2668,  1.4102]],
│           
│                   [[ 0.4277,  0.1447,  0.2535],
│                    [-0.7681,  0.2668,  1.4102]],
│           
│                   [[ 0.4277,  0.1447,  0.2535],
│                    [-0.7681,  0.2668,  1.4102]],
│           
│                   [[ 0.4277,  0.1447,  0.2535],
│                    [-0.7681,  0.2668,  1.4102]],
│           
│                   [[ 0.4277,  0.1447,  0.2535],
│                    [-0.7681,  0.2668,  1.4102]],
│           
│                   [[ 0.4277,  0.1447,  0.2535],
│                    [-0.7681,  0.2668,  1.4102]],
│           
│                   [[ 0.4277,  0.1447,  0.2535],
│                    [-0.7681,  0.2668,  1.4102]],
│           
│                   [[ 0.4277,  0.1447,  0.2535],
│                    [-0.7681,  0.2668,  1.4102]]])
└── 'x' --> <FastTreeValue 0x7f76b4947b50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.3 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f778f112a90>
├── 'a' --> tensor([[ 0.4277,  0.1447,  0.2535],
│                   [-0.7681,  0.2668,  1.4102],
│                   [ 0.4277,  0.1447,  0.2535],
│                   [-0.7681,  0.2668,  1.4102],
│                   [ 0.4277,  0.1447,  0.2535],
│                   [-0.7681,  0.2668,  1.4102],
│                   [ 0.4277,  0.1447,  0.2535],
│                   [-0.7681,  0.2668,  1.4102],
│                   [ 0.4277,  0.1447,  0.2535],
│                   [-0.7681,  0.2668,  1.4102],
│                   [ 0.4277,  0.1447,  0.2535],
│                   [-0.7681,  0.2668,  1.4102],
│                   [ 0.4277,  0.1447,  0.2535],
│                   [-0.7681,  0.2668,  1.4102],
│                   [ 0.4277,  0.1447,  0.2535],
│                   [-0.7681,  0.2668,  1.4102]])
└── 'x' --> <FastTreeValue 0x7f76b4e63130>
    └── 'c' --> tensor([[-2.1334,  2.4800,  1.1582,  0.9062],
                        [ 0.3769, -1.3441,  0.0103, -0.9183],
                 

In [23]:
%timeit t_cat(trees)

42 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

98.7 µs ± 4.91 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-2.1334,  2.4800,  1.1582,  0.9062],
                       [ 0.3769, -1.3441,  0.0103, -0.9183],
                       [ 1.2914,  0.7107, -0.4273, -0.5815]],
              
                      [[-2.1334,  2.4800,  1.1582,  0.9062],
                       [ 0.3769, -1.3441,  0.0103, -0.9183],
                       [ 1.2914,  0.7107, -0.4273, -0.5815]],
              
                      [[-2.1334,  2.4800,  1.1582,  0.9062],
                       [ 0.3769, -1.3441,  0.0103, -0.9183],
                       [ 1.2914,  0.7107, -0.4273, -0.5815]],
              
                      [[-2.1334,  2.4800,  1.1582,  0.9062],
                       [ 0.3769, -1.3441,  0.0103, -0.9183],
                       [ 1.2914,  0.7107, -0.4273, -0.5815]],
              
                      [[-2.1334,  2.4800,  1.1582,  0.9062],
                       [ 0.3769, -1.3441,  0.0103, -0.9183],
                       [ 1.2914,  0.7107, -0.4273, -0.5815]],

In [26]:
%timeit Batch.stack(batches)

115 µs ± 5.81 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-2.1334,  2.4800,  1.1582,  0.9062],
                      [ 0.3769, -1.3441,  0.0103, -0.9183],
                      [ 1.2914,  0.7107, -0.4273, -0.5815],
                      [-2.1334,  2.4800,  1.1582,  0.9062],
                      [ 0.3769, -1.3441,  0.0103, -0.9183],
                      [ 1.2914,  0.7107, -0.4273, -0.5815],
                      [-2.1334,  2.4800,  1.1582,  0.9062],
                      [ 0.3769, -1.3441,  0.0103, -0.9183],
                      [ 1.2914,  0.7107, -0.4273, -0.5815],
                      [-2.1334,  2.4800,  1.1582,  0.9062],
                      [ 0.3769, -1.3441,  0.0103, -0.9183],
                      [ 1.2914,  0.7107, -0.4273, -0.5815],
                      [-2.1334,  2.4800,  1.1582,  0.9062],
                      [ 0.3769, -1.3441,  0.0103, -0.9183],
                      [ 1.2914,  0.7107, -0.4273, -0.5815],
                      [-2.1334,  2.4800,  1.1582,  0.9062],
                   

In [28]:
%timeit Batch.cat(batches)

198 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

642 µs ± 25.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
